In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys,inspect
sys.path.insert(0,"..")

In [3]:
from math import log10, sqrt
import cv2
import numpy as np

import pytorch_ssim
import torch
from torch.autograd import Variable

In [4]:
def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    f = dir.split('/')[-1].split('_')[-1]
    #print (dir, f)
    dirs= os.listdir(dir)
    for img in dirs:

        path = os.path.join(dir, img)
        #print(path)
        images.append(path)
    return images

def read_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (192, 256))
    return image


def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 10 * log10(max_pixel / sqrt(mse))
    return psnr

In [5]:
exp_name = "rob_test"

In [6]:
gt = '../outputs/{}/person'.format(exp_name)
a1 = '../outputs/{}/acgpn'.format(exp_name)
a2 = '../outputs/{}/fifa'.format(exp_name)

In [7]:
a2

'../outputs/rob_test/fifa'

In [9]:
ssim_scores = []
psnr_scores = []


gtp = sorted(make_dataset(gt))
a1p = sorted(make_dataset(a1))

for x,y in zip(gtp, a1p):
    img1 = read_image(x)
    img2 = read_image(y)
    
    print(img1.shape, img2.shape)
    
    img1 = np.expand_dims(img1 / 255.0, axis=0)
    img2 = np.expand_dims(img2 / 255.0, axis=0)
    
    # Compute PSNR
    psnr = PSNR(img1, img2)
    psnr_scores.append(psnr)
    
    #if torch.cuda.is_available():
    img1 = torch.from_numpy(img1)
    img2 = torch.from_numpy(img2)
    score = pytorch_ssim.ssim(img1, img2)
    score = score.item()
    ssim_scores.append(score)
        
        
sscore = np.array(ssim_scores).mean()
print("SSIM score: {:.4f}".format(sscore))

pscore = np.array(psnr_scores).mean()
print("PSNR score: {:.4f}".format(pscore))

(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
SSIM score: 0.9276
PSNR score: 32.3156


In [10]:
# 0.9276 

# 0.9529

In [11]:
!pip install pytorch_fid

  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.1-py3-none-any.whl size=14835 sha256=0df4ed63b30572fe289dc4ec61abaeff89d32a74553f802ca295c2e0fc4c325d
  Stored in directory: /home/hasib/.cache/pip/wheels/df/c8/a0/cce2ed7671ae52be132ae836e429bba6148544f83b7962b4bc
Successfully built pytorch-fid


In [15]:
!python -m pytorch_fid '../outputs/rob_test/person' '../outputs/rob_test/fifa'

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.44it/s]
FID:  147.62776817946724


In [ ]:
# 152.43189
# 147.62776

| Model  | Test all SSIM (higher) | Test easy SSIM (higher) | Test medium SSIM (higher)| Test hard SSIM (higher)| Test FID (lower) |
| ------------- | ------------- | ------------- | ------------- | ------------- |------------- |
| ACGPN 5000  | 0.8231 | 0.8311 |0.8214 |0.8044 |26.167 |
| ResUnet 32filt, BN (resunet_g1, ngf=32) | **0.8273** | **0.8356** | **0.8254** | **0.8082** | **23.032** |
| ACGPN 5000 N  | 0.8256 |- |- | - | 24.43| 
| ResUnet 32filt, BN (resunet_g1, ngf=32)N| 0.8288 | - | - | - | 24.647